# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096787


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:05<01:30,  2.91s/it]

Rendering models:   9%|▉         | 3/33 [00:07<01:13,  2.44s/it]

Rendering models:  12%|█▏        | 4/33 [00:07<00:51,  1.78s/it]

Rendering models:  15%|█▌        | 5/33 [00:07<00:37,  1.36s/it]

...

Rendering models:  91%|█████████ | 30/33 [00:16<00:01,  2.87it/s]

Rendering models:  94%|█████████▍| 31/33 [00:16<00:00,  3.10it/s]

Rendering models:  97%|█████████▋| 32/33 [00:17<00:00,  3.06it/s]

Rendering models: 100%|██████████| 33/33 [00:17<00:00,  3.27it/s]

not-logged-in-3b0b929f4475fd276561    0.001729
matiasd123                            0.001110
not-logged-in-553c4a543f16d42b2154    0.002271
not-logged-in-f737655ff84da201cfb7    0.000932
c196948                               0.002691
jverderame                            0.014429
jnarayanbvg                           0.001931
Quarkgc                               0.000943
not-logged-in-e7932b1588574fe4dae2    0.053482
not-logged-in-3371c656e2057627cca0    0.329007
not-logged-in-6cd9487f1f13f15d1751    0.002484
astro_lab_ncmns                       0.006143
not-logged-in-2908c5b97527b9f7279b    0.022479
not-logged-in-6c3c85931d83a1164a0d    0.002127
Galaxy-one                            0.000600
casbee                                0.001226
lattecuc                              0.000552
sn346808                              0.035593
sn346808                              0.040554
sn346808                              0.022682
sn346808                              0.022748
Rmarks1701   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())